In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

HIGHD_DIR = "D:\\AppliedGroupCW3\\Collaberate\\highD-dataset-master\\Python\\data"


def load_highd_data(recording_id):
    # tracks metadata
    tracks_meta_path = os.path.join(HIGHD_DIR, f"{recording_id}_tracksMeta.csv")
    tracks_meta = pd.read_csv(tracks_meta_path)
    
    # tracks data
    tracks_path = os.path.join(HIGHD_DIR, f"{recording_id}_tracks.csv")
    tracks = pd.read_csv(tracks_path)
    
    # recording metadata
    recording_meta_path = os.path.join(HIGHD_DIR, f"{recording_id}_recordingMeta.csv")
    recording_meta = pd.read_csv(recording_meta_path)
    
    return tracks_meta, tracks, recording_meta


def calculate_traffic_measures_multilane(tracks_df, tracks_meta_df, driving_direction=1, section_length=420):
    direction_map = tracks_meta_df[['id', 'drivingDirection']].set_index('id').to_dict()['drivingDirection']
    tracks_df = tracks_df.copy()  
    tracks_df['drivingDirection'] = tracks_df['id'].map(direction_map)

    direction_tracks = tracks_df[tracks_df['drivingDirection'] == driving_direction].copy() 
    
    if len(direction_tracks) == 0:
        print(f"Warning: No data found for direction {driving_direction}")
        return pd.DataFrame()
    
    
    max_frame = direction_tracks['frame'].max()
    
    results = []
    
    for frame in range(int(max_frame) + 1):

        frame_data = direction_tracks[direction_tracks['frame'] == frame].copy()  
        if len(frame_data) == 0:
            continue
            
        vehicle_count = len(frame_data)
        
        lane_count = len(frame_data['laneId'].unique())
        if lane_count == 0:
            continue
        
        # density in veh/m 
        density = vehicle_count / section_length

        # original speed in m/s
        frame_data.loc[:, 'speed_ms'] = frame_data['xVelocity'].abs()
        
        # convert to mph 
        frame_data.loc[:, 'speed_mph'] = frame_data['speed_ms'] * 2.23694
        
        # Calculate average speed in mph
        avg_speed_mph = frame_data['speed_mph'].mean()
        avg_speed_ms = frame_data['speed_ms'].mean()

        # flow is density * speed in m/s 
        flow = density * avg_speed_ms
        
        results.append({
            'frame': frame,
            'time': frame / 25.0,  
            'density': density,
            'flow': flow,
            'avg_speed_ms': avg_speed_ms,
            'avg_speed_mph': avg_speed_mph,
            'lane_count': lane_count
        })
    
    return pd.DataFrame(results)

def create_density_flow_diagram(measures_df, title=""):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    x = measures_df['density'].values
    y = measures_df['flow'].values
    

    speeds = measures_df['avg_speed_mph'].values
    scatter = ax.scatter(x, y, c=speeds, s=50, alpha=0.5, cmap='viridis')
    plt.colorbar(scatter, label='Speed (mph)')

    z = np.polyfit(x, y, 2)
    p = np.poly1d(z)
    
    x_fit = np.linspace(0, max(x) * 1.1, 100)
    y_fit = p(x_fit)
    
    y_fit = np.maximum(y_fit, 0)
    
    ax.plot(x_fit, y_fit, "r--", linewidth=2, label='Quadratic Fit')
    
    equation = f"flow = {z[0]:.6f}*density² + {z[1]:.6f}*density + {z[2]:.6f}"
    ax.text(0.05, 0.95, equation, transform=ax.transAxes, 
             verticalalignment='top', fontsize=12, bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    ax.set_xlabel('Density (veh/m)', fontsize=14)
    ax.set_ylabel('Flow (veh/s)', fontsize=14)
    ax.set_title(f'Multi-lane Density-Flow Relationship {title}', fontsize=16)
    ax.grid(True, linestyle='--', alpha=0.7)
    
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=0)
    
    plt.legend()
    return fig, ax

def analyze_multilane_direction(recording_id, driving_direction=1, section_length=420):
    tracks_meta, tracks, _ = load_highd_data(recording_id)

    measures_df = calculate_traffic_measures_multilane(
        tracks, tracks_meta, driving_direction, section_length
    )
    
    
    title = f"Recording {recording_id}, Direction {driving_direction}"
    fig, ax = create_density_flow_diagram(measures_df, title)
    
    plt.savefig(f"multilane_density_flow_highd_{recording_id}_dir{driving_direction}.png", 
               dpi=300, bbox_inches='tight')
    plt.show()
    

    print(f"Average density: {measures_df['density'].mean():.6f} veh/m")
    print(f"Average flow: {measures_df['flow'].mean():.6f} veh/s")
    print(f"Average speed: {measures_df['avg_speed_mph'].mean():.2f} mph ({measures_df['avg_speed_ms'].mean():.2f} m/s)")
    print(f"Number of lanes: {measures_df['lane_count'].max()}")
    
    return measures_df


measures_df = analyze_multilane_direction("01", driving_direction=1)